In [14]:
import pyspark
from pyspark.sql import SparkSession, types
from pyspark.sql import functions as F
from datetime import datetime

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/03 14:56:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
schema = types.StructType(
    [types.StructField('_c0', types.StringType(), True), 
     types.StructField('_c1', types.IntegerType(), True), 
     types.StructField('_c2', types.IntegerType(), True), 
     types.StructField('_c3', types.IntegerType(), True), 
     types.StructField('_c4', types.IntegerType(), True), 
     types.StructField('_c5', types.LongType(), True), 
     types.StructField('_c6', types.IntegerType(), True), 
     types.StructField('_c7', types.IntegerType(), True)]
)

In [4]:
df = spark.read \
    .schema(schema) \
    .csv('custom_1988_2020.csv')

In [5]:
df.repartition("_c0")

DataFrame[_c0: string, _c1: int, _c2: int, _c3: int, _c4: int, _c5: bigint, _c6: int, _c7: int]

In [8]:
df.write \
    .mode("overwrite") \
    .parquet('data/', )

24/01/03 14:58:48 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/01/03 14:59:06 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/01/03 14:59:07 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/01/03 14:59:24 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/01/03 14:59:24 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/01/03 14:59:39 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/01/03 14:59:39 WARN MemoryManager: Total allocation exceeds 95.00% 

In [38]:
df = spark.read \
    .parquet('data/')

In [41]:
def date_format(date_str):
    date_obj = datetime.strptime(date_str, '%Y%m')
    formatted_date = date_obj.strftime('%Y-%m')
    return formatted_date
date_format_udf = F.udf(date_format, returnType=types.StringType())

In [42]:
df \
    .withColumn('clean_date', F.to_date(date_format_udf(df._c0))) \
    .select('clean_date', '_c0') \
    .show(5)

+----------+------+
|clean_date|   _c0|
+----------+------+
|2007-10-01|200710|
|2007-10-01|200710|
|2007-10-01|200710|
|2007-10-01|200710|
|2007-10-01|200710|
+----------+------+
only showing top 5 rows

